In [1]:
import cv2
import numpy as np

# Load YOLOv3 weights and configuration files
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load class names
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set input and output layers
layer_names = net.getLayerNames()
output_layer_names = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Initialize camera
cap = cv2.VideoCapture(0)

# Loop through frames from the camera
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Resize the frame to a smaller size
    frame = cv2.resize(frame, (640, 480))
    
    # Get height, width, and channels of the frame
    height, width, channels = frame.shape
    
    # Prepare frame for inference
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Pass frame through the network
    net.setInput(blob)
    outs = net.forward(output_layer_names)

    # Process detections
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw boxes and labels on the frame
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = classes[class_ids[i]]
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            text = f"{label} {confidence:.2f}"
            cv2.putText(frame, text, (x, y-5), font, 2, color, 2)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) == ord('0'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()